In [1]:
from coniii import *
from process_input import fish_input_process


C:\Users\Riickyyy\anaconda3\envs\coniii\lib\site-packages\coniii\samplers.py:381: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def iterate_neighbors(n,ix,expdJ,r):
C:\Users\Riickyyy\anaconda3\envs\coniii\lib\site-packages\coniii\samplers.py:2342: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def cross(vec1, vec2):


ModuleNotFoundError: No module named 'process_input'

In [ ]:
TIME = 3600

In [ ]:
file = 'samples.csv'
import pandas as pd
#read in data
data = pd.read_csv(file, header=None)

print(data.shape)


samples = fish_input_process(file,time=TIME)

print(samples.shape)
FISH_N = samples.shape[0]# neuro
FISH_N_PAIRS = int(FISH_N * (FISH_N - 1) / 2)

In [ ]:
samples.shape[0]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6), dpi=80)
plt.imshow(samples, aspect="auto", cmap="gray_r", interpolation="none")

In [ ]:
calc_e, calc_observables, mchApproximation = define_ising_helper_functions()
print(calc_e)
print(calc_observables)
print(mchApproximation)

In [ ]:
solver = ClusterExpansion(samples.T)

multipliers, ent, clusters, deltaSdict, deltaJdict= solver.solve(threshold=0.1, full_output=True)

In [ ]:
J = multipliers[FISH_N:]

# 创建一个 N*N 的零矩阵
matrix = np.zeros((FISH_N, FISH_N))

# # 使用数组的前N个元素填充对角线
# matrix[np.diag_indices(FISH_N)] = multipliers[:FISH_N]

# 使用数组的后N+2(N*N-1)个元素填充上半矩阵
index = FISH_N
for i in range(FISH_N):
    for j in range(i+1, FISH_N):
        matrix[i, j] = multipliers[index]
        index += 1
# 获取上半矩阵（包含对角线）
upper_matrix = np.triu(matrix)

# 转置上半矩阵并将其添加到下半部分
lower_matrix = np.transpose(upper_matrix)
lower_matrix = np.tril(lower_matrix, -1)

# 将上半矩阵和转置的下半矩阵相加得到完整的矩阵
final_matrix = upper_matrix + lower_matrix

In [ ]:
#plot imshow and add lines
plt.imshow(final_matrix, aspect="auto", cmap="gray")
ax = plt.gca() # 获取当前的坐标轴
ax.set_xticks([x-0.5 for x in range(1, final_matrix.shape[1])], minor=True) # 设置x轴的小刻度
ax.set_yticks([y-0.5 for y in range(1, final_matrix.shape[0])], minor=True) # 设置y轴的小刻度
plt.grid(which='minor', linestyle='-', linewidth=2)
plt.colorbar()